# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
%pip install sentence-transformers
%pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [2]:
K = 25

In [3]:
df_news = pd.read_csv('data/df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [4]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [6]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [7]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [8]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [9]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [10]:
from google.colab import userdata

openai_key = userdata.get('OPEN_AI_KEY')

In [12]:
from openai import OpenAI

In [15]:
client = OpenAI(api_key=openai_key)

In [16]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

As the moon shone brightly, the little unicorn whispered her dreams to the stars, and with a sprinkle of stardust, they turned into vibrant flowers that bloomed in her magical forest.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [17]:
import yfinance as yf

In [28]:
ticker_meta = {
    'TICKER': [],
    'COMPANY_NAME': [],
    'SECTOR': [],
    'INDUSTRY': []
}

for ticker in df_news['TICKER'].unique():
    yf_ticker = yf.Ticker(ticker)
    ticker_meta['TICKER'].append(ticker)
    ticker_meta['COMPANY_NAME'].append(yf_ticker.info['longName'])
    ticker_meta['SECTOR'].append(yf_ticker.info['sector'])
    ticker_meta['INDUSTRY'].append(yf_ticker.info['industry'])

In [29]:
df_meta = pd.DataFrame(ticker_meta)
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [31]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [32]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [37]:
class RAGPipeline:
    def __init__(self, faiss_store, client):
        self.faiss_store = faiss_store
        self.client = client

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [doc for doc, _, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    @staticmethod
    def _generate_prompt_context(documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, _, _ in documents:
            context += f"- {doc}\n"

        return context

In [38]:
rag = RAGPipeline(faiss_store, client)

In [46]:
def display_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### Sources used for generating this answer:\n"

    docs_md = [f"- {doc}" for doc in docs]
    result += "\n".join(docs_md)

    print_markdown(result)

In [49]:
print_markdown("# Topic-focused questions")

for question in questions_topic:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news reveals several major concerns about inflation:

1. **Persistent Inflation**: The Federal Reserve's minutes indicate growing apprehension regarding persistent inflation, suggesting it could impede economic growth. This concern signals that inflation may not be a temporary issue and could require sustained policy actions.

2. **Food Inflation**: Specific emphasis has been placed on food inflation, which is notably affecting consumer prices and dampening expectations for potential interest rate cuts. The rising costs associated with food are creating additional pressure on households and could lead to broader economic implications.

3. **Economic Slowdown**: There are fears that persistent inflation could lead to an economic slowdown. As inflation continues to rise, it may hinder consumer spending and business investment, ultimately affecting overall economic growth.

4. **Tariff Impacts**: Ongoing changes in tariffs have added complexity to inflation concerns, particularly impacting food prices. Fluctuations in trade policies can exacerbate shortages or increase costs, further influencing inflation rates.

These points illustrate a landscape where inflation remains a critical concern for policymakers, consumers, and investors alike.

### Sources used for generating this answer:
- Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines appears to be mixed, with a notable focus on both optimism and skepticism regarding various stocks. Here's a summary based on the articles referenced:

1. **Ambitious Price Targets**: Several articles highlight that Wall Street has set high price targets for certain stocks. While this indicates potential upside, there's an underlying skepticism about these forecasts due to institutional pressures on analysts, which may lead to overly optimistic projections.

2. **Positive Market Performance**: Some stocks are currently outperforming the market, driven by favorable developments like new products or strong community support on platforms like Reddit. This points to a more bullish sentiment among certain investor segments.

3. **Caution with Analyst Ratings**: There is a consistent caution expressed regarding analyst ratings. The sentiment suggests that investors should be wary of taking forecasts at face value due to analysts' incentives to maintain positive outlooks for business reasons, particularly in environments like mergers and acquisitions.

4. **Bearish Indicators**: When Wall Street adopts a bearish stance on a stock, it signals heightened concern. This sentiment is especially important as such ratings are relatively rare, reflecting a cautious outlook from analysts.

5. **Bullish Yet Questionable Fundamentals**: There is a strong bullish sentiment on certain stocks, despite questionable fundamentals, which highlights a potential disconnect between market enthusiasm and underlying financial health.

Overall, while there's an evident bullish sentiment in some areas, significant skepticism remains, urging investors to approach forecasts and ratings with caution.

### Sources used for generating this answer:
- 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.
- 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.
- 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.
- 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.
- 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.

## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is playing a transformative role in the finance sector, as highlighted by several recent news stories. Here are some key themes:

1. **Productivity and Efficiency**: Companies like Jack Henry & Associates are integrating AI-driven lending technologies to boost productivity and reduce human error in financial operations. This reflects a broader trend where AI is utilized to streamline processes, enhance accuracy, and improve service delivery in the financial sector.

2. **Investment Insights**: Major tech firms like Meta Platforms are focusing on AI investments that could substantially impact their valuations. Investors are increasingly looking at companies' AI capabilities, suggesting that AI is becoming a crucial factor in stock performance assessments.

3. **Stock Market Dynamics**: Names like Palantir and Upstart are in the spotlight as billionaires invest in these AI-focused firms. Analysts predict significant upside potential for these stocks, indicating that AI-driven solutions—such as Upstart's credit risk assessment tool—are reshaping investment strategies and valuations.

4. **Generative AI Applications**: Companies like Palantir are helping both government and commercial clients integrate generative AI into their operations. This not only enhances functionality in various sectors but also stimulates demand for AI solutions, contributing to stock price increases despite market volatility.

5. **Content and Strategic Innovation**: Beyond traditional finance applications, generative AI is influencing sectors like entertainment, as seen with Netflix. The ability to produce and refine content rapidly signifies that AI's role is expanding even beyond core financial functions into adjacent industries.

Overall, AI is emerging as a crucial component driving innovation, investment strategies, and operational efficiency in the finance realm and beyond.

### Sources used for generating this answer:
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation.
- Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

In [50]:
print_markdown("# Company-focused questions")

for question in questions_company:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
Microsoft is not directly mentioned in the provided articles, but the context around AI investments suggests a competitive atmosphere among major tech companies. Here's how Microsoft might be portrayed based on the overall trends in AI investments as indicated by the news:

1. **Strategic Focus**: Given that other companies, like Salesforce and Meta, are making headlines for their AI efforts, Microsoft likely positions itself as a serious contender in the AI space. Its investments could be portrayed as strategic moves aimed at boosting its market value and competitiveness.

2. **Innovation and Leadership**: If Microsoft’s AI initiatives are effective, the news may highlight the company as a leader in innovation, contributing significantly to advancements in AI technology and applications. This portrayal could contrast with companies that are criticized for over-investing or mismanaging their AI strategies.

3. **Market Response**: Investors and analysts might view Microsoft's AI investments with optimism, especially if they lead to improved productivity or new revenue streams. Such perception can positively influence Microsoft's stock performance.

4. **Comparative Analysis**: In a landscape filled with companies investing heavily in AI, Microsoft could be analyzed in comparison to other tech giants. Coverage might highlight its unique strategies or successful implementations of AI technologies.

In summary, while not explicitly covered, Microsoft is likely to be seen as a competitive player in the AI realm, potentially being depicted as a leader in strategy and innovation, especially if its initiatives yield significant results.

### Sources used for generating this answer:
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here.
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.
- Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […]

## Q: What financial news headlines connect Amazon with automation or logistics?
Here are some financial news headlines that connect Amazon with automation or logistics:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article discusses how Amazon is integrating artificial intelligence across its services, which includes logistics and automation to enhance overall operational efficiency.

2. **Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead**: The focus on how Amazon's performance is being enhanced through advanced technologies, including automation in logistics.

These headlines illustrate Amazon's commitment to leveraging automation and AI to optimize its logistics and operational capabilities.

### Sources used for generating this answer:
- Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]
- Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."
- Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi
- Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).
- Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).

In [51]:
print_markdown("# Industry-focused questions")

for question in questions_industry:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
The emerging themes in recent financial news about the semiconductor industry include:

1. **Investor Focus on Revenue Trends**: There is a strong emphasis on international revenue trends for companies like ON Semiconductor Corp., highlighting how these trends impact Wall Street forecasts and stock performance.

2. **Market Sentiment and Earnings Reports**: Despite some companies reporting soft earnings, there is a sense of optimism among shareholders, indicating a possible disconnect between earnings performance and stock price reaction.

3. **Stock Selection and Potential**: Articles are highlighting specific semiconductor stocks that are gaining attention from investors, including those considered to have significant upside potential, as exemplified by coverage of ON Semiconductor in the context of billionaire investors' stock picks.

4. **Evaluation During Earnings Season**: The end of earnings seasons is seen as a pivotal time for assessing company performance and making informed investment decisions in the semiconductor sector, as analysts look for winners among various semiconductor stocks.

5. **Sector Competitiveness**: There is ongoing evaluation of how companies like Vishay Intertechnology and others in the analog semiconductor space are faring within a competitive market landscape.

These themes indicate a blend of optimism, cautious analysis, and ongoing scrutiny of financial performance as the semiconductor industry navigates economic challenges and opportunities.

### Sources used for generating this answer:
- Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.
- ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.
- Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...
- Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.
- ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]

## Q: What trends are being reported in the retail industry?
Recent trends in the retail industry highlight various challenges and adaptations as retailers navigate a quickly changing landscape:

1. **Shift in Business Models**: Retailers are adjusting their business strategies in response to technological advancements that alter consumer shopping behaviors. This includes an increased emphasis on e-commerce and digital platforms.

2. **Volatile Demand**: Consumer spending is fluctuating, leading to uncertainty in the retail sector. Over the past six months, retail stocks have seen a decline of 13.7%, significantly worse than the broader S&P 500, which dropped only 5.5%. This reflects the cyclical nature of consumer discretionary spending.

3. **Impact of Trade Policies**: Retailers have faced challenges due to trade tensions and tariffs, particularly in terms of supply chain adjustments and price increases. Many retailers are proactively managing their supply chains to mitigate the effects of legal and policy changes.

4. **Economic Sensitivity**: The performance of the retail sector is closely tied to economic conditions. The recent downturn in consumer discretionary sales, with a reported drop of 12.3% over six months, further underscores the sensitivity of this sector to economic cycles.

Overall, the retail industry is marked by a transitional phase influenced by technology, consumer behavior, and external economic factors, leading to increased volatility in stock performance.

### Sources used for generating this answer:
- 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.
- Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.
- 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.
- Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...
- Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news highlights several risks and challenges facing the energy industry:

1. **Legislative Risks**: A potential bill advancing in Congress poses significant threats to the renewable energy sector by repealing critical subsidies. This could render many renewable projects uneconomical, leading to a sharp decline in renewable energy stocks and investment.

2. **Market Volatility**: The oilfield services sector, including major companies like SLB, HAL, and BKR, is grappling with declining oil prices and increasing tariffs. These factors, combined with shrinking drilling budgets, create uncertainty in the industry’s future profitability and sustainability.

3. **Trade Conflicts**: The ongoing trade tensions, particularly those related to tariffs, are further complicating the landscape. Companies are facing challenges in supply chains and pricing pressures, which could affect operational costs and competitiveness.

4. **Economic Factors**: The interplay between renewable energy initiatives and traditional fossil fuel markets is precarious, particularly as investors weigh the risks associated with fluctuating commodity prices, tariffs, and changing regulations.

Overall, the energy industry is navigating a complex environment influenced by legislative actions, market dynamics, and geopolitical developments.

### Sources used for generating this answer:
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

YOUR WRITTEN RESPONSE HERE


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

YOUR WRITTEN RESPONSE HERE
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

YOUR WRITTEN RESPONSE HERE


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

YOUR WRITTEN RESPONSE HERE
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

YOUR WRITTEN RESPONSE HERE
  

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [88]:
class EnrichedRAGPipeline:
    def __init__(self, faiss_store, client, company_metadata):
        self.faiss_store = faiss_store
        self.client = client
        self.company_metadata = company_metadata

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [(doc, self._get_company_metadata(meta['TICKER'])) for doc, meta, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.
        This added context will take the form of recent news article related to
        the question. Each of these articles will be about a specific
        company, sector or industry.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    def _generate_prompt_context(self, documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, meta, _ in documents:
            ticker = meta['TICKER']
            c_meta = self._get_company_metadata(ticker)
            context += f"This following news article is related to the '{c_meta['COMPANY_NAME']}' company ({ticker}), operating in the {c_meta['SECTOR']} sector and the {c_meta['INDUSTRY']} industry."
            context += f"> {doc}\n"

        return context

    def _get_company_metadata(self, ticker):
        return self.company_metadata[self.company_metadata['TICKER'] == ticker].to_dict(orient='records')[0]

In [89]:
enriched_rag = EnrichedRAGPipeline(faiss_store, client, df_meta)

In [90]:
def display_enriched_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### The following news articles were used for generating this answer:\n"
    result += "| Ticker | Company | Sector | Industry | Article |\n"
    result += "|--------|--------------|--------|----------|-----|\n"
    for doc, meta in documents:
        result += f"| {meta['TICKER']} | {meta['COMPANY_NAME']} | {meta['SECTOR']} | {meta['INDUSTRY']} | {doc} |\n"
    print_markdown(result)

In [91]:
print_markdown("# Topic-focused questions")
for question in questions_topic:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news highlights several major concerns regarding inflation:

1. **Persistent Inflation Risks**: Articles related to BlackRock indicate that there are increasing worries about ongoing inflation in the U.S., particularly following insights from the Federal Reserve’s May policy meeting. The Fed's recognition of these inflation risks suggests a potential for economic slowdowns as they consider monetary policy adjustments.

2. **Food Inflation Impact**: Reports mentioning Tesla, Lululemon, and Broadcom emphasize the significant impact of food inflation on consumer spending and broader economic conditions. This has dampened expectations for potential interest rate cuts, signaling that inflation remains a primary hurdle for economic recovery and consumer confidence.

3. **Tariff Uncertainties**: The influence of tariffs on inflation is another concern, as ongoing adjustments could contribute to fluctuating prices, further complicating the economic landscape. This is particularly relevant for companies across various sectors, including technology and consumer goods, which are sensitive to supply chain shifts.

Overall, there's a shared anxiety about the persistence of inflation, how it affects monetary policy, and the broader implications for economic growth and consumer sentiment.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| BLK | BlackRock, Inc. | Financial Services | Asset Management | Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown. |
| TSLA | Tesla, Inc. | Consumer Cyclical | Auto Manufacturers | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| NVDA | NVIDIA Corporation | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| LULU | lululemon athletica inc. | Consumer Cyclical | Apparel Retail | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| AVGO | Broadcom Inc. | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |


## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines can be characterized as cautiously optimistic, but with notable skepticism regarding the motivations behind analysts' stock ratings. Here are some key takeaways from the articles:

1. **Ambitious Price Targets**: Several articles highlight Wall Street's ambitious price targets for various stocks, suggesting an attractive upside potential. However, there's a cautionary note that analysts may face institutional pressures, leading to overly optimistic forecasts. This indicates a degree of skepticism about whether these targets will be met.

2. **Outperformance Amid Catalysts**: Some stocks are outperforming the market due to positive catalysts, such as new product launches or favorable news cycles. This reflects a positive sentiment toward certain sectors like technology, but investors are encouraged to remain vigilant.

3. **Bullish Sentiments with a Warning**: While Wall Street shows overwhelming bullishness on certain stocks, there is an underlying doubt about their fundamentals. Analysts are noted for rarely issuing sell ratings, which can skew investor perception and create a potentially misleading sense of security.

4. **Bearish Observations**: In contrast, some analysts are turning bearish on specific stocks, highlighting that there are stocks that deserve more caution. This indicates a sector of the market where investor confidence is waning, suggesting a more nuanced sentiment landscape where not all equities are viewed positively.

Overall, while there is a segment of investor optimism buoyed by potential growth and favorable conditions, a layer of skepticism persists, urging investors to critically evaluate analyst opinions and market conditions before making investment decisions.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts. |
| MCHP | Microchip Technology Incorporated | Technology | Semiconductors | 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase. |
| MPWR | Monolithic Power Systems, Inc. | Technology | Semiconductors | 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory. |
| DRI | Darden Restaurants, Inc. | Consumer Cyclical | Restaurants | 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory. |
| RVTY | Revvity, Inc. | Healthcare | Diagnostics & Research | 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover. |


## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is playing a transformative role across various sectors in recent finance-related news stories, with several companies harnessing its capabilities to enhance productivity, improve decision-making, and innovate their service offerings. Here are some key highlights:

1. **Jack Henry & Associates, Inc. (JKHY)**: This company has integrated AI-driven lending technology with Algebrik, showcasing how AI can streamline financial operations. This integration aims to increase productivity and decrease human error in lending processes, highlighting AI's role in improving efficiency and accuracy in financial services.

2. **Meta Platforms, Inc. (META)**: Meta is making significant investments in AI applications, aiming to leverage these technologies to enhance its core business offerings. Investors view these AI investments as potential growth drivers, suggesting that AI could significantly contribute to the future valuation of Meta's stock beyond its traditional business model.

3. **Palantir Technologies Inc. (PLTR)**: Palantir is actively tapping into the demand for AI solutions among government and commercial clients. Their AI Platform (AIP) assists in integrating generative AI capabilities into operations, which can lead to improved efficiencies and insights. The stock has seen positive momentum, suggesting that the market recognizes the potential growth associated with their AI endeavors.

4. **Netflix, Inc. (NFLX)**: Netflix is exploring generative AI to enhance content production, indicating the technology's potential to streamline creative processes and boost productivity in the entertainment sector. This approach not only aids in content creation but could also enhance viewer engagement and satisfaction.

In summary, AI is being recognized as a crucial factor in driving innovation, efficiency, and growth across various industries, including finance, technology, and entertainment. Companies that effectively integrate AI into their operations are likely to see significant competitive advantages and opportunities for value creation.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago. |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |


In [92]:
print_markdown("# Company-focused questions")
for question in questions_company:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
The articles provided focus on various companies involved in the artificial intelligence (AI) space, but they don't specifically mention Microsoft. However, we can infer Microsoft's portrayal in AI news through broader trends in the tech industry that are reflected in these articles.

1. **Investment in AI**: Similar to other tech giants like Meta, Microsoft is heavily invested in AI technology. Companies that effectively leverage AI to enhance their products or services are often seen positively by investors. 

2. **Market Positioning**: The discussions around companies like Salesforce indicate that firms must balance AI investments with other business areas to maintain overall health. If Microsoft is perceived as successfully integrating AI while continuing to support its core offerings (like cloud services or productivity software), it would likely be viewed favorably.

3. **Competitive Landscape**: The increased focus on AI by various companies, including Meta and Netflix, shows a competitive race in technology where companies must innovate to keep up. If Microsoft is portrayed as a leader or as having a strong position in AI, this would enhance investor confidence.

4. **User Adoption and Functionality**: If news articles highlight Microsoft’s AI tools (such as those integrated into Office products or Azure cloud services) reaching significant user milestones or enhancing productivity, this would further bolster its positive portrayal.

Overall, while the specifics about Microsoft's representation in the news are not detailed in these articles, the themes suggest that companies like Microsoft are seen as key players in the AI field, and they are likely benefiting from the ongoing AI competition and investment narrative in the tech industry.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| CRM | Salesforce, Inc. | Technology | Software - Application | How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here. |
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […] |


## Q: What financial news headlines connect Amazon with automation or logistics?
Here are some financial news headlines connecting Amazon with automation or logistics:

1. **Amazon's AI Roadmap With AWS CEO Garman**: Matt Garman, CEO of Amazon Web Services, discusses how every aspect of Amazon is leveraging artificial intelligence, which underpins their logistics and operations strategy.

2. **Truist Reiterates Buy on Amazon.com as Q2 Revenue Tracks Ahead**: The emphasis on Amazon's integration of AI reveals its commitment to automating processes, further enhancing its logistics capabilities.

3. **Research Reports Featuring Amazon.com**: These reports highlight Amazon's position in the market, where its focus on automation and logistics could drive future growth and efficiency in its operations.

These headlines illustrate Amazon's strategic focus on artificial intelligence and automation to streamline its logistics and improve operational efficiencies.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […] |
| AMZN | Amazon.com, Inc. | Consumer Cyclical | Internet Retail | Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology." |
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi |
| AON | Aon plc | Financial Services | Insurance Brokers | Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS). |
| CHRW | C.H. Robinson Worldwide, Inc. | Industrials | Integrated Freight & Logistics | Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW). |


In [93]:
print_markdown("# Industry-focused questions")
for question in questions_industry:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
The recent financial news regarding the semiconductor industry reveals several emerging themes:

1. **International Revenue Trends**: There is a growing emphasis on how companies within the semiconductor sector, like ON Semiconductor, are achieving revenue from international markets. This is becoming a crucial factor for investors as it impacts Wall Street forecasts and the strategic directions of these firms.

2. **Investor Sentiment**: There is noticeable investor attention on key players such as ON Semiconductor, suggesting a speculative interest in the stock market potential of these companies. Investors are keen to examine how earnings are faring and what these trends indicate for future performance.

3. **Earnings Performance**: The semiconductor sector seems to be experiencing mixed earnings results, with some companies, like ON Semiconductor, facing soft earnings without significantly affecting shareholder confidence. This indicates resilience and potentially foundational strength despite challenging economic conditions.

4. **Stock Selections by Influential Investors**: The focus on selections made by prominent investors, such as Billionaire Glenn Russell Dubin, reflects a broader trend of market sentiment and trust in certain semiconductor stocks. This might highlight potential growth areas in the industry, suggesting that analysts and investors are looking for stocks with substantial upside potential.

5. **Market Assessment Post-Earnings Season**: The end of earnings season has triggered evaluations of stock performance within the industry, offering opportunities for investors to identify strong candidates, similar to how Analog Devices and Vishay Intertechnology performed recently. This reflects a strategic approach among investors to reassess portfolios based on current financial health.

Overall, the semiconductor industry is navigating through a mix of international growth potential, earnings performance evaluation, and investor sentiment, with continuous observation of stock opportunities driven by industry leaders.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week... |
| ADI | Analog Devices, Inc. | Technology | Semiconductors | Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […] |


## Q: What trends are being reported in the retail industry?
The retail industry is currently experiencing several noteworthy trends:

1. **Adapting Business Models**: Retailers are increasingly modifying their business strategies to align with changing consumer behaviors driven by technological advancements. This includes adopting digital platforms and enhancing online shopping experiences.

2. **Volatile Demand**: Consumer spending remains unpredictable, with recent data indicating that retail stocks have decreased by 13.7% over the past six months. This decline has been sharper than the S&P 500's 5.5% loss, indicating significant challenges in the sector.

3. **Impact of Trade Policies**: Retailers are actively adjusting their supply chains to mitigate issues stemming from trade wars and tariffs. Executives are reporting that price increases resulting from these changes have already affected product pricing on shelves.

4. **Economic Sensitivity**: The performance of the consumer discretionary sector, which includes companies like Hilton Worldwide, has been closely tied to economic cycles. Recent evidence suggests that demand trends may not be favorable, as highlighted by a 12.3% decline in this sector over six months, worse than the S&P 500’s 2.1% decline.

These trends reflect a challenging landscape for retailers as they navigate shifts in consumer behavior and macroeconomic factors.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss. |
| BBY | Best Buy Co., Inc. | Consumer Cyclical | Specialty Retail | Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves. |
| HLT | Hilton Worldwide Holdings Inc. | Consumer Cyclical | Lodging | 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline. |
| PKG | Packaging Corporation of America | Consumer Cyclical | Packaging & Containers | Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst... |
| APD | Air Products and Chemicals, Inc. | Basic Materials | Specialty Chemicals | Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to... |


## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news highlights several significant risks and challenges facing the energy industry:

1. **Legislative Changes Affecting Renewables**: There is growing concern about a bill in Congress that could repeal crucial subsidies for the renewable energy sector. This legislative move poses a serious risk to the economic viability of renewable energy projects, as it may render many of them uneconomical just as companies have ramped up production and electricity generation. Companies like NextEra Energy and Enphase Energy, which operate in the renewable space, are particularly vulnerable to these changes.

2. **Fluctuating Oil Prices**: For companies involved in oil and gas services, such as Halliburton and Baker Hughes, there is heightened uncertainty due to sliding oil prices. These fluctuations can lead to reduced drilling budgets, which directly impacts revenues and growth potential for service companies.

3. **Rising Tariffs**: The increased tariffs on materials and equipment used in the energy sector compound the financial pressures faced by oilfield service companies. The combination of rising costs and shrinking budgets raises concerns about profitability and operational sustainability.

4. **Market Demand and Competition**: The energy sector is also grappling with the challenges of maintaining demand. The shift towards liquefied natural gas (LNG) and the integration of AI in operations may provide some support, but the overall outlook remains uncertain, particularly as companies adjust to these market conditions.

These challenges illustrate a landscape filled with legislative, market, and operational risks, making the future of different sectors within the energy industry quite complex.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| NEE | NextEra Energy, Inc. | Utilities | Utilities - Regulated Electric | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| ENPH | Enphase Energy, Inc. | Technology | Solar | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| HAL | Halliburton Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| BKR | Baker Hughes Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| FCX | Freeport-McMoRan Inc. | Basic Materials | Copper | 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty. |


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE